In [2]:
import tensorflow as tf
import numpy as np

In [94]:
# generate 2 layer network with 
# m features on the input, n nodes on the first layer, and 1 value as output
n=2
m=3
x = tf.Variable(tf.random.normal((m, 1)), name='x')
W = tf.Variable(tf.random.normal((n, m)), name='W')
y = tf.Variable(tf.random.normal(shape=[n]), name='y')
diag_y = tf.linalg.diag(y)
u = tf.random.uniform(shape=[1,n])
# print(x,W)
# print(y,diag_y)

# 1, Using auto-differentiation from deep learning framework

In [95]:
with tf.GradientTape() as tape:
    f= u@diag_y@W@x
df_dx = tape.gradient(f, x)
print(df_dx)

tf.Tensor(
[[ 1.2352295 ]
 [-0.5007099 ]
 [-0.79972357]], shape=(3, 1), dtype=float32)


# 2, Using the analytical solution given by Jacobi’s formula

- v = (zeta, lam, tau)
- **T1** = diag(lam)
- **T2** = diag(tau)

| matix name | shape |
| ------- | ---- |
| **tau** | m |
| **lam** | n |
| **T1** | n,n |
| **T2** | m,m |
| **W** | n,m |
| **u** | 1,n |
| **M**, **M_zeta**, **M_c**| 1+m+n+n+m,1+m+n+n+m |
| **M_tau**| m,1+m+n+n+m,1+m+n+n+m |
| **M_lam**| n,1+m+n+n+m,1+m+n+n+m |


In [96]:
M_zeta = np.zeros([1+m+n+n+m,1+m+n+n+m])
M_zeta[0,0] = 1
M_tau = np.zeros([m,1+m+n+n+m,1+m+n+n+m])
print(np.shape(M_tau))
for i in range(m):
    M_tau[i][0][0] = -1
    M_tau[i][i+1][i+1] = 1
    M_tau[i][i+1+m+n+n][i+1+m+n+n] = 1

(3, 11, 11)


In [97]:
M_lam = np.zeros([n,1+m+n+n+m,1+m+n+n+m])
print(np.shape(M_lam))
for i in range(n):
    for j in range (m):
        M_lam[i][1+m+i][1+j] = -W[i][j]
        M_lam[i][1+j][1+m+i] = -W[i][j]
    M_lam[i][1+m+i][1+m+i] = 2
    M_lam[i][1+m+n+i][1+m+n+i] = 1
# print(M_lam[1])

(2, 11, 11)


In [98]:
M_c = np.zeros([1+m+n+n+m,1+m+n+n+m])
for i in range(n):
    M_c[1+m+i][1] = -u[0][i]
    M_c[1][1+m+i] = -u[0][i]
# print(M_c)